In [2]:
''' Do not change this cell '''
#Functions

def check_gpu():
  if tf.test.gpu_device_name() != '':
    print("GPU sucessfully connected")
  else: print("PLease connect GPU")

In [3]:
#import all the required packages and  libraries for FineTuning LLM
!pip install transformers datasets accelerate bitsandbytes trl
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import tensorflow as tf

# Check if TensorFlow can access the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using the following GPU(s):")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU found. TensorFlow will run on CPU.")


TensorFlow is using the following GPU(s):
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [5]:
''' Do not change this cell '''
check_gpu()

GPU sucessfully connected


In [6]:
ds = load_dataset("jizzu/llama2_indian_law_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/900k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24607 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/276 [00:00<?, ? examples/s]

In [7]:
modeL_name='NousResearch/Llama-2-7b-chat-hf'

In [8]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 24607
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 455
    })
    test: Dataset({
        features: ['text'],
        num_rows: 276
    })
})


In [9]:

print(ds['train'][0])


{'text': "###Human:\nWhat is the difference between a petition and a plaint in Indian law?\n###Assistant:\nA petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases."}


In [25]:
def reformat_to_llama2(conversation_text):
    segments = conversation_text.split('###')

    # Initialize a list to store the reformatted segments
    reformatted_segments = []

    # Iterate over pairs of segments (Human and Assistant)
    for i in range(1, len(segments), 2):
        # Extract and clean up the human text
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there's a corresponding assistant text
        if i + 1 < len(segments):
            assistant_text = segments[i + 1].strip().replace('Assistant:', '').strip()

            # Apply the Llama 2 template for both human and assistant
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # In case there's no assistant text, handle the human segment alone
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    # Join all the reformatted segments and return the result
    return ' '.join(reformatted_segments)

def reformat_dataset(dataset):
    # Apply the reformatting to all examples in the dataset
    def transform_example(example):
        return {'text': reformat_to_llama2(example['text'])}

    # Apply the function to the entire dataset
    formatted_dataset = dataset.map(transform_example, batched=False)

    return formatted_dataset

# Reformat the 'train' split of the dataset
formatted_dataset = reformat_dataset(ds['train'])
formatted_test=reformat_dataset(ds['test'])
formatted_validation=reformat_dataset(ds['validation'])

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

In [26]:
print(formatted_dataset[0])

{'text': "<s>[INST] What is the difference between a petition and a plaint in Indian law? [/INST] A petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases. </s>"}


In [27]:
pip install datasets huggingface_hub


In [30]:
!pip install datasets --upgrade # Upgrade datasets library to the latest version
from huggingface_hub import login
repo_id = "22deshmukh/indian-law-dataset"
login(token="hf_XiwftNezeXngfDnvxxxxxOrRrefuycAy")

# Now push the dataset to the Hub
# Remove repo_type and filename arguments
# The dataset will be saved with the default name (e.g., "train_data.arrow")
formatted_dataset.push_to_hub(repo_id)
formatted_test.push_to_hub(repo_id, split="test") # Specify split for test data
formatted_validation.push_to_hub(repo_id, split="validation") # Specify split for validation data

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/277 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/374 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/22deshmukh/indian-law-dataset/commit/9a41a133820aff2bc2d068890688273a966fb48a', commit_message='Upload dataset', commit_description='', oid='9a41a133820aff2bc2d068890688273a966fb48a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/22deshmukh/indian-law-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='22deshmukh/indian-law-dataset'), pr_revision=None, pr_num=None)